#Install and setup

##1. Install Unsloth & Load Model

In [ ]:
# @title 1. Install Unsloth & Load Model
# Installs unsloth and loads Llama-3-8B-Instruct (4-bit) specifically optimized for Colab

# 1. Install Unsloth (specifically for Colab)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# 2. Import & Load Model
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Roman Urdu is token-heavy, but 2048 is safe for Colab
dtype = None # Auto detection
load_in_4bit = True # Essential for Colab T4 GPU

# We use Llama 3.1 8B Instruct (Best balance of size/performance)
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

print(f"🔄 Loading {model_name}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Add LoRA Adapters (The Fine-Tuning Layer)
# We target "all-linear" modules to help it learn the new language patterns better
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank: 16 is standard. 32 or 64 is better for new languages but uses more VRAM.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Optimized to 0 for Unsloth
    bias = "none",    # Optimized to "none" for Unsloth
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✅ Model Loaded & LoRA Adapters Attached!")

#Dataformatting and Training

##2. Format Data & Start Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title 2. Format Data & Start Training
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset

# 1. Load your Dataset again
dataset = load_dataset("Redgerd/roman-urdu-alpaca-qa-mix", split = "train")

# 2. Define the Chat Format (Alpaca -> Llama 3 Prompt)
# This teaches the model: "Here is an instruction, here is the context, give me the answer."
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Fill the template
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched = True)

# 3. Set up the Trainer (Updated for 500 Steps + Google Drive Save)
print("🚀 Starting Trainer Setup...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,

        # --- KEY CHANGES HERE ---
        max_steps = 500,        # Increased to 500 for better quality
        save_steps = 100,       # Save a backup every 100 steps
        output_dir = "/content/drive/My Drive/RomanUrduModel", # Save directly to Drive
        # ------------------------

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
    ),
)

# 4. Train!
print("🔥 Training Started!")
# trainer_stats = trainer.train()
# New line to resume from your specific checkpoint
print("New line to resume from my specific checkpoint\n")
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/RomanUrduModel/checkpoint-500")

Now, the model is saved on drive

#Testing after training

Inferences and testing is done in seperate colab file : https://colab.research.google.com/drive/1ob1OQgX7OoTNNAUGn1nsfywOx8y7Yc7h?usp=sharing

##4. Launch Public Web App (Gradio)

In [ ]:
# # @title 4. Launch Public Web App (Gradio)
# # This creates a shareable link to your model
# !pip install -q gradio

# import gradio as gr
# from transformers import TextStreamer

# # 1. Enable Native Fast Inference
# FastLanguageModel.for_inference(model)

# def chat_response(message, history):
#     # 'message' is the user's new question
#     # 'history' is the chat context (we are ignoring it for simple QA to save memory,
#     # but you can add it if you want multi-turn memory)

#     # Prepare the prompt
#     prompt = alpaca_prompt.format(
#         message, # instruction
#         "",      # input (optional context)
#         "",      # output - leave blank
#     )

#     # Tokenize and move to GPU
#     inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

#     # Generate
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens = 256,
#         use_cache = True
#     )

#     # Decode and clean up (remove the prompt from the answer)
#     decoded_output = tokenizer.batch_decode(outputs)[0]
#     cleaned_response = decoded_output.split("### Response:")[-1].strip().replace("<|end_of_text|>", "")

#     return cleaned_response

# # 2. Build the Interface
# demo = gr.ChatInterface(
#     fn=chat_response,
#     title="🤖 My Roman Urdu AI",
#     description="Ask me anything in Roman Urdu! (e.g., 'Biryani kaise banate hain?')",
#     examples=["Pakistan ka capital kya hai?", "Biryani ki recipe batao", "Lahore kyun mashhoor hai?"],
#     theme="soft"
# )

# # 3. Launch with Public Link
# # share=True creates the public URL
# print("🚀 Launching... Click the link that ends in '.gradio.live' below!")
# demo.launch(share=True, debug=True)

#Save to GGUF (16-bit)

Code is continue in the same testing/inferences colab file : https://colab.research.google.com/drive/1ob1OQgX7OoTNNAUGn1nsfywOx8y7Yc7h?usp=sharing

In [ ]:
# # @title 5. Save Model to GGUF (For Deployment)
# # Ye code aapke model ko ek single file (model.gguf) mein convert karke
# # Google Drive mein save kar dega.

# # 1. Save to GGUF (16-bit)
# # Ye file thori bari hogi lekin quality best hogi.
# if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# # 2. Save to GGUF (8-bit Quantized) - RECOMMENDED
# # Ye file size mein choti hogi aur fast chalegi.
# print("💾 Saving model to Google Drive as GGUF...")
# model.save_pretrained_gguf(
#     "/content/drive/My Drive/RomanUrduModel/model_q8", # Folder in Drive
#     tokenizer,
#     quantization_method = "q8_0" # 8-bit quantization
# )
# print("✅ DONE! Check your Google Drive folder for 'model_q8.gguf'")

#Move code from drive to hugging face

For this code is on testing/inferences colab file : https://colab.research.google.com/drive/1ob1OQgX7OoTNNAUGn1nsfywOx8y7Yc7h?usp=sharing

In [ ]:
# # @title 6. Push Model to Hugging Face Hub
# from huggingface_hub import login

# # 1. Login (Paste your token when asked)
# # Yahan apna "Write" token paste karein
# hf_token = "I will add it here" # <--- Paste Token Here
# login(token=hf_token)

# # 2. Upload to Hugging Face
# username = "Aapka_HF_Username" # <--- Apna Hugging Face username likhein
# model_name = "Roman-Urdu-Llama-3"     # Model ka naam jo rakhna hai

# print("☁️ Uploading model to Hugging Face...")
# # Ye GGUF (8-bit) wala upload karega jo sabse acha hai
# model.push_to_hub_gguf(
#     f"{username}/{model_name}",
#     tokenizer,
#     quantization_method = "q8_0",
#     token = hf_token
# )

# print(f"✅ DONE! Model is live at: https://huggingface.co/{username}/{model_name}")